# Prepare nice json competenframeworksfor Flask app

In [2]:
import json
import requests
import pprint

base_url = 'http://141.5.108.59:3000/ims/case/v1p0'

In [3]:
r = requests.get(base_url + '/CFDocuments')

list_of_documents = []

for item in r.json()['CFDocuments']:
   list_of_documents.append(item['identifier'])  

list_of_documents

['1e2537a8-3b92-11ea-800f-0242ac1a0003',
 '05a9d6f8-3b96-11ea-bc94-0242ac1a0003']

## Get JSON for each document

In [13]:
competence_frameworks = []

for item in list_of_documents:

    r = requests.get(base_url + '/CFPackages/' + item)

    competence_framework = []
    for item in r.json()['CFAssociations']:
        if item['associationType'] == 'isChildOf':
            competence_framework.append(
                (item['destinationNodeURI']['identifier'],(item['originNodeURI']['identifier']))
                )

    parents, children = zip(*competence_framework)
    root_nodes = {x for x in parents if x not in children}
    for node in root_nodes:
        print(node)
        r = requests.get('http://141.5.108.59:3000/ims/case/v1p0/CFDocuments/' + node)
        print(r.json())
        str_title = r.json()['creator']
        print(str_title)
        competence_framework.append(('Root', node))
        competence_framework.append(('Title', r.json()['creator']))

    def get_nodes(node):
        d = {}
        try:
            if node == "Root":
                d['identifier'] = r.json()['creator']
            else:
                d['identifier'] = node
            r = requests.get('http://141.5.108.59:3000/ims/case/v1p0/CFItems/' + d['identifier'])

            if r.status_code == 404:
                # this might be a package URI
                r = requests.get('http://141.5.108.59:3000/ims/case/v1p0/CFDocuments/' + d['identifier'])
                d['title'] = r.json()['title']
                d['creator'] = r.json()['creator']
                d['language'] = r.json()['language']
            else:
                # this is a Item
                d['title'] = r.json()['humanCodingScheme']
                d['educationLevel'] = r.json()['educationLevel']
                d['fullStatement'] = r.json()['fullStatement']

        except:
            pass
        children = get_children(node)
        if children:
            d['nodes'] = [get_nodes(child) for child in children]
        return d

    def get_children(node):
        return [x[1] for x in competence_framework if x[0] == node]

    tree = get_nodes('Root')
    competence_frameworks.append(tree)

pprint.pprint(competence_frameworks)

1e2537a8-3b92-11ea-800f-0242ac1a0003
{'uri': 'http://141.5.108.59:3000/uri/1e2537a8-3b92-11ea-800f-0242ac1a0003', 'identifier': '1e2537a8-3b92-11ea-800f-0242ac1a0003', 'lastChangeDateTime': '2020-01-20T14:35:34+00:00', 'CFPackageURI': {'title': 'Bildungsstandards im Fach Deutsch für den Primarbereich', 'identifier': '1e2537a8-3b92-11ea-800f-0242ac1a0003', 'uri': 'http://141.5.108.59:3000/uri/p1e2537a8-3b92-11ea-800f-0242ac1a0003'}, 'creator': 'Kultusministerkonferenz', 'title': 'Bildungsstandards im Fach Deutsch für den Primarbereich', 'language': 'de'}
Kultusministerkonferenz
05a9d6f8-3b96-11ea-bc94-0242ac1a0003
{'uri': 'http://141.5.108.59:3000/uri/05a9d6f8-3b96-11ea-bc94-0242ac1a0003', 'identifier': '05a9d6f8-3b96-11ea-bc94-0242ac1a0003', 'lastChangeDateTime': '2020-01-20T15:19:09+00:00', 'CFPackageURI': {'title': 'Teilrahmenplan Deutsch', 'identifier': '05a9d6f8-3b96-11ea-bc94-0242ac1a0003', 'uri': 'http://141.5.108.59:3000/uri/p05a9d6f8-3b96-11ea-bc94-0242ac1a0003'}, 'creator': 'M

## Add relation informations to items

In [17]:
for item in list_of_documents:

    r = requests.get(base_url + '/CFPackages/' + item)

    associations = []
    for item in r.json()['CFAssociations']:
        # for now I don't want child associations
        if item['associationType'] != 'isChildOf':
            association = {
                'associationType': item['associationType'],
                'originNodeURI': item['originNodeURI'],
                'destinationNodeURI': item['destinationNodeURI']
            }
            associations.append(association)
    pprint.pprint(associations)

[]
[{'associationType': 'isRelatedTo',
  'destinationNodeURI': {'identifier': '2d553cfe-3b93-11ea-9ddf-0242ac1a0003',
                         'title': 'Destination node',
                         'uri': 'http://141.5.108.59:3000/uri/2d553cfe-3b93-11ea-9ddf-0242ac1a0003'},
  'originNodeURI': {'identifier': '88756154-3b98-11ea-8d52-0242ac1a0003',
                    'title': 'Sprechen und Zuhören',
                    'uri': 'http://141.5.108.59:3000/uri/88756154-3b98-11ea-8d52-0242ac1a0003'}},
 {'associationType': 'exactMatchOf',
  'destinationNodeURI': {'identifier': '6cba87aa-3b93-11ea-97b6-0242ac1a0003',
                         'title': 'Destination node',
                         'uri': 'http://141.5.108.59:3000/uri/6cba87aa-3b93-11ea-97b6-0242ac1a0003'},
  'originNodeURI': {'identifier': 'e19009ce-3b98-11ea-93ae-0242ac1a0003',
                    'title': 'Gespräche führen',
                    'uri': 'http://141.5.108.59:3000/uri/e19009ce-3b98-11ea-93ae-0242ac1a0003'}},
 {'assoc

In [18]:
competence_frameworks = []

for item in list_of_documents:

    r = requests.get(base_url + '/CFPackages/' + item)

    competence_framework = []
    associations = []
    for item in r.json()['CFAssociations']:
        if item['associationType'] == 'isChildOf':
            competence_framework.append(
                (item['destinationNodeURI']['identifier'],(item['originNodeURI']['identifier']))
                )
        else:
            association = {
                'associationType': item['associationType'],
                'originNodeURI': item['originNodeURI'],
                'destinationNodeURI': item['destinationNodeURI']
            }
            associations.append(association)

    parents, children = zip(*competence_framework)
    root_nodes = {x for x in parents if x not in children}
    for node in root_nodes:
        print(node)
        r = requests.get('http://141.5.108.59:3000/ims/case/v1p0/CFDocuments/' + node)
        print(r.json())
        str_title = r.json()['creator']
        print(str_title)
        competence_framework.append(('Root', node))
        competence_framework.append(('Title', r.json()['creator']))

    def get_nodes(node):
        d = {}
        try:
            if node == "Root":
                d['identifier'] = r.json()['creator']
            else:
                d['identifier'] = node
            r = requests.get('http://141.5.108.59:3000/ims/case/v1p0/CFItems/' + d['identifier'])

            if r.status_code == 404:
                # this might be a package URI
                r = requests.get('http://141.5.108.59:3000/ims/case/v1p0/CFDocuments/' + d['identifier'])
                d['title'] = r.json()['title']
                d['creator'] = r.json()['creator']
                d['language'] = r.json()['language']
            else:
                # this is a Item
                d['title'] = r.json()['humanCodingScheme']
                d['educationLevel'] = r.json()['educationLevel']
                d['fullStatement'] = r.json()['fullStatement']
            for item in associations:
                if item['originNodeURI']['identifier'] == d['identifier']:
                    d['associationType'] = item['associationType']
                    d['destinationNodeURI'] = item['destinationNodeURI']
        except:
            pass
        children = get_children(node)
        if children:
            d['nodes'] = [get_nodes(child) for child in children]
        return d

    def get_children(node):
        return [x[1] for x in competence_framework if x[0] == node]

    tree = get_nodes('Root')
    competence_frameworks.append(tree)

pprint.pprint(competence_frameworks)

1e2537a8-3b92-11ea-800f-0242ac1a0003
{'uri': 'http://141.5.108.59:3000/uri/1e2537a8-3b92-11ea-800f-0242ac1a0003', 'identifier': '1e2537a8-3b92-11ea-800f-0242ac1a0003', 'lastChangeDateTime': '2020-01-20T14:35:34+00:00', 'CFPackageURI': {'title': 'Bildungsstandards im Fach Deutsch für den Primarbereich', 'identifier': '1e2537a8-3b92-11ea-800f-0242ac1a0003', 'uri': 'http://141.5.108.59:3000/uri/p1e2537a8-3b92-11ea-800f-0242ac1a0003'}, 'creator': 'Kultusministerkonferenz', 'title': 'Bildungsstandards im Fach Deutsch für den Primarbereich', 'language': 'de'}
Kultusministerkonferenz
05a9d6f8-3b96-11ea-bc94-0242ac1a0003
{'uri': 'http://141.5.108.59:3000/uri/05a9d6f8-3b96-11ea-bc94-0242ac1a0003', 'identifier': '05a9d6f8-3b96-11ea-bc94-0242ac1a0003', 'lastChangeDateTime': '2020-01-20T15:19:09+00:00', 'CFPackageURI': {'title': 'Teilrahmenplan Deutsch', 'identifier': '05a9d6f8-3b96-11ea-bc94-0242ac1a0003', 'uri': 'http://141.5.108.59:3000/uri/p05a9d6f8-3b96-11ea-bc94-0242ac1a0003'}, 'creator': 'M